# **VGG Pretrained**
---
---

## **Necessary Imports**

In [1]:
from tensorflow import keras
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

## **Download Data From Drive**

In [2]:
import gdown
url = "https://drive.google.com/file/d/12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP/view?usp=sharing"
file_id = url.split("/")[-2]
print(file_id)
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "catdog.zip")

12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP


Downloading...
From: https://drive.google.com/uc?/export=download&id=12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP
To: /content/catdog.zip
100%|██████████| 9.09M/9.09M [00:00<00:00, 120MB/s]


'catdog.zip'

In [3]:

!unzip catdog.zip

Archive:  catdog.zip
replace train/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## **Imports for Pretrain data**

In [4]:
from tensorflow import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

## **Set the path to your training and validation data**

In [5]:
train_data_dir = '/content/train'
validation_data_dir = '/content/validation'

## **Set the number of training and validation samples**

In [6]:
num_train_samples = 2000
num_validation_samples = 800


## **Set the number of epochs and batch size**

In [7]:
epochs = 5
batch_size = 16

## **Load the VGG16 model without the top layer**

In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#include_top = False : Ignores the top 3 Fully Connected Layer after Flatten()

## **Freeze the base model layers**

In [9]:
for layer in base_model.layers:
    layer.trainable = False

## **Create a new model**

In [10]:
model = Sequential()

# Add the base model as a layer
model.add(base_model)

# Add custom layers on top of the base model
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


## **Compile the model**

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## **Preprocess the training and validation data**

In [12]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


## **Train the model**

In [13]:
model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size)


Epoch 1/5
 22/125 [====>.........................] - ETA: 15:06 - loss: 7.5478 - accuracy: 0.8309

125/125 [==============================] - 238s 2s/step - loss: 7.5478 - accuracy: 0.8309 - val_loss: 3.0679 - val_accuracy: 0.8983


## **Save the trained model**

In [14]:
model.save('dog_cat_classifier.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
